[A Neural Network in 11 lines of Python (Part 1) - i am trask](http://iamtrask.github.io/2015/07/12/basic-python-network/)

In [2]:
import numpy as np

In [8]:
X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])
y = np.array([[0,1,1,0]]).T

syn0 = 2*np.random.random((3,4)) - 1
syn1 = 2*np.random.random((4,1)) - 1

for j in range(60000):
    l1 = 1/(1+np.exp(-(np.dot(X,syn0))))
    l2 = 1/(1+np.exp(-(np.dot(l1,syn1))))
    l2_delta = (y-l2)*(l2*(1-l2))
    l1_delta = l2_delta.dot(syn1.T) * (l1*(1-l1))
    syn1 += l1.T.dot(l2_delta)
    syn0 += X.T.dot(l1_delta)

In [9]:
y.shape

(4, 1)

In [5]:
X.shape

(4, 3)

## Part1: A Tiny Toy Network

In [22]:
import numpy as np


def  nonlin(x, deriv=False):
    if (deriv == True):
        return x*(1-x)
    return 1/(1+np.exp(-x))


X = np.array([  [0,0,1], 
                [0,1,1], 
                [1,0,1], 
                [1,1,1]  ])


y = np.array([[0,0,1,1]]).T



np.random.seed(1)
# 权重矩阵大 size 由输入和输出的个数确定，也可以用 l0 和 l1 的 size 确定
# 初始化权重 W 矩阵
syn0 = 2*np.random.random((3,1)) - 1

for iter in range(10000):
    # l1 是预测值
    # full batch (4 个 samples 一次用上)
    l0 = X
    l1 = nonlin(np.dot(l0,syn0))
    
    # 误差
    l1_error = y - l1
    
    # (y-l1) * l1* (1-l1)；误差 × 导数（ sigmoid 的导函数：y(1-y)
    # 更新的系数
    l1_delta = l1_error * nonlin(l1, True)
    
    # 根据输入 X 和 更新系数，更新权重 W 矩阵；y 和 l1 越接近（预测的越准），系数权重越不需要更新（其实可以理解为更新的是输入的 X）
    syn0 += np.dot(l0.T,l1_delta)
        
print (l1)

[[ 0.00966449]
 [ 0.00786506]
 [ 0.99358898]
 [ 0.99211957]]


In [20]:
l1_error.shape

(4, 1)

In [21]:
nonlin(l1,True).shape

(4, 1)

In [12]:
l1_delta.shape

(4, 1)

In [13]:
syn0.shape

(3, 1)

In [14]:
l1.shape

(4, 1)

In [19]:
2*np.random.random((3,1)) - 1

array([[ 0.34093502],
       [-0.1653904 ],
       [ 0.11737966]])

## A Slightly Harder Problem

In [28]:
import numpy as np

def nonlin(x,deriv=False):
    if (deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

X = np.array([  [0,0,1], 
                [0,1,1], 
                [1,0,1], 
                [1,1,1]  ])

y = np.array([[0,1,1,0]]).T

np.random.seed(1)

# 随机初始化 W （零均值）
syn0 = 2*np.random.random((3,4)) - 1
syn1 = 2*np.random.random((4,1)) - 1

for j in range(60000):
    
    # Feed forward through layers 0, 1, and 2
    # X full_batch 4*3；如果是 single，那就是 1*3，表示 3 个神经元在 l0 也就是输入层
    # l1 4*4；如果是 single，那就是 1*4，表示 4 个神经元在 l1 也就是隐层 1
    # l2 4*1；如果是 single，那就是 1*1，表示 1 个神经元在 l2 也就是输出层
    l0 = X
    l1 = nonlin(np.dot(l0, syn0))
    l2 = nonlin(np.dot(l1, syn1))
    
    # how much did we miss the target value?
    # l2 4*1；le_error 4*1；如果是 single，那就是 1*1，表示 1 个神经元，也就是一个输出（预测值）和训练集的误差
    l2_error = y - l2
    
    if (j%10000) == 0:
        print ("Error:" + str(np.mean(np.abs(l2-l2_error))))
    
    # 反向传播，矩阵都是反过来乘：
    # 开始反向传播！
    # 先从输出层开始！
    
    # in what direction is the target value?
    # were we really sure? if so, don't change too much
    # l2_delta 4*1；如果是 single，也就是 1*1，表示更新的大小；来自 输出层 error
    # l2_delta 更新的是 l1_error 和 syn1，也就是说更新了隐层的误差和隐层与输出层之间的权重矩阵；它自己在输出层，往回退
    # 这玩意儿可以理解为更新后的 l2
    l2_delta = l2_error * nonlin(l2, deriv=True) # = l2_error * (l2 * (1-l2)) 导数（梯度）× error
    
    
    # --------------------------***-------------------
    
    # 反向传播计算 隐层 Error！
    
    # l1_error 4*4；如果是 single，也就是 1*4，也就是 隐层 输出值的误差
    l1_error = l2_delta.dot(syn1.T)
    
    # 反向传播更新！
    
    # in what direction is the target l1?
    # were we really sure? if so, don't change too much
    # l1_delta 4*4；如果是 single，也就是 1*4，表示更新的大小；来自 隐层 error
    # l1_delta 更新的是 syn0，也就是说更新了输入层和隐层之间的权重矩阵，l0_error 不存在，l0 是输入的 X；它自己在隐层，往回退
    # 这玩意儿就可以理解为更新后的 l1
    l1_delta = l1_error * nonlin(l1, deriv=True)
    
    # 调整权重矩阵 W
    # 隐层和输出层之间的调整，syn1 (4*4)*(4*1)=4*1；如果是 single，那也是 (4*1)*(1*1)=4*1，每个值均做了调整
    # 输入层和隐层之间的调整，syn0 (3*4)*(4*4)=3*4；如果是 single，那也是 (3*1)*(1*4)=3*4，每个值均做了调整
    # 权重矩阵 shape 不变！
    # 用更新后的 l2, l1 更新权重矩阵
    syn1 += l1.T.dot(l2_delta)
    syn0 += l0.T.dot(l1_delta)
    
    # -----------***-------------
    # 正向（前向）传播时，l1 = nonlin(np.dot(l0, syn0))
    # 反向（后向）传播时，l0.dot(syn0) = l0.dot(l0.T).dot(l1_delta)
    # l0.dot(l0.T) 是一个方阵，与 l1_delta 一起对 l1 本身做了调整（迭代调整）

Error:0.536660920155
Error:0.500331174315
Error:0.500206235232
Error:0.500159662229
Error:0.500134253787
Error:0.500117871151
